# 크롤링 코드

In [4]:
import openpyxl
import os
import sys
import time
import json
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd
import time
from bs4 import BeautifulSoup

def collect_menus(name, driver,result):
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 첫 메뉴 목록 찾기
    menu_lists = soup.select('.list_menu > li')
    menu_list=[]
    menu_detail_list=[]
    customer=[]
    
    for menu in menu_lists:
        
        menu_name = menu.select('.loss_word')[0].text
        menu_detail = menu.select('.txt_menu')# 리뷰
       
        
        menu_list.append(menu_name)
        menu_detail_list.append(menu_detail)
        customer.append(name)
        
    tmp_df = pd.DataFrame({'식당':customer,'메뉴이름':menu_list,'상세정보':menu_detail_list})

    return pd.concat([result, tmp_df])

def crawling_menus(df):
    options = webdriver.ChromeOptions()
    # options.add_argument('headless')
    #options.add_argument("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36   ")
    options.add_argument('lang=ko_KR')
    driver = webdriver.Chrome(options=options)  # chromedriver 열기
    driver.set_window_position(0, 0)
    driver.set_window_size(1000, 3000)
    driver.get('https://map.kakao.com/')
    
    result = pd.DataFrame({'식당':[1],'메뉴이름':[1],'상세정보':[1]})
    
    for i in range(len(df)):
        gu_name= df['지역'][i]
        rs_name = df['식당이름'][i]
        
        name = gu_name +" " + rs_name
        
        driver.find_element(By.XPATH,'//*[@id="search.keyword.query"]').clear()
        
        search_area = driver.find_element(By.XPATH,'//*[@id="search.keyword.query"]')
        #검색 단어 
        search_area.send_keys(name)

        driver.find_element(By.XPATH,'//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)
        
        time.sleep(3)

    
        #해당 음식점 창 열기
        search = driver.find_element(By.XPATH, '//*[@data-id="moreview"]')
        search.send_keys(Keys.ENTER)

        time.sleep(2)

        #열어진 창에서 작업
        driver.switch_to.window(driver.window_handles[-1])

        time.sleep(2) #

        #메뉴 가져오는 함수
        result = collect_menus(name, driver, result)

        driver.close()

        driver.switch_to.window(driver.window_handles[-1])

    return result[1:]


## 레시피랑 매칭 작업

In [5]:
def zacard(A,B):
    intersection_cardinality = len(set.intersection(*[set(A), set(B)]))
    union_cardinality = len(set.union(*[set(A), set(B)]))
    similar = intersection_cardinality / float(union_cardinality)
    
    return similar

def find_ingred(menu_name,recipe):
    main_score=0
    main_menu=""
    
    for menu in recipe['음식']:
        tmp_score = zacard(menu, menu_name)
        
        if tmp_score > main_score:
            main_score = tmp_score
            main_menu = menu
    return main_menu

def matching_menu_ingred(crawled_menu, recipe):
    #레시피 불러오기
    #adr=""
    #recipe = pd.read_csv(adr+'통합레시피_유니크.csv',encoding = 'euc-kr')
    ingred_list =[]

    for i in range(len(crawled_menu)):
    
        tmp1 = find_ingred(crawled_menu['메뉴이름'][i],recipe)
        

        ingred_list.append(recipe[recipe['음식']==tmp1]['재료'].item())

    ingred_list=pd.DataFrame({'재료':ingred_list})
    crawled_menu['재료'] = ingred_list
    return crawled_menu

def crawling_and_matching(df, recipe):
    crawled = crawling_menus(df)
    
    result =matching_menu_ingred(crawled, recipe)
    
    return result

# 예시

In [8]:
df= pd.DataFrame({'지역': ["서울 동작구 성대로 24"],'식당이름':['여기로']})
recipe = pd.read_csv('통합레시피_유니크2.csv',encoding='euc-kr')

cw_mn = crawling_menus(df)

In [9]:
cw_mn

,식당,메뉴이름,상세정보
0,서울 동작구 성대로 24 여기로,고르곤졸라꿀피자,[]
1,서울 동작구 성대로 24 여기로,시저샐러드,[]
2,서울 동작구 성대로 24 여기로,부추골뱅이,[]
3,서울 동작구 성대로 24 여기로,낙지볶음,[]
4,서울 동작구 성대로 24 여기로,찹스테이크,[]


In [100]:
#예시 데이터
df= pd.DataFrame({'지역': ["서울 동작구 성대로 24"],'식당이름':['여기로']})
recipe = pd.read_csv('통합레시피_유니크2.csv',encoding='euc-kr')

#crawled_menu = crawling_menus(df)
#matching_menu_ingred(crawled_menu, recipe)

menu_and_ingred = crawling_and_matching(df, recipe)
menu_and_ingred

,식당,메뉴이름,상세정보,재료
0,서울 동작구 성대로 24 여기로,고르곤졸라꿀피자,[],"{'시판용', '간마늘', '시금치또띠아', '홍고추', '바질', '배', '후추..."
1,서울 동작구 성대로 24 여기로,시저샐러드,[],"{'로메인', '중간크기', '바질', '후추', '디죤', '마요네즈', '플레인..."
2,서울 동작구 성대로 24 여기로,부추골뱅이,[],"{'참기름', '고추장', '설탕', '붉은고추', '식초', '매실', '매실액'..."
3,서울 동작구 성대로 24 여기로,낙지볶음,[],"{'국수', '간마늘', '고추장', '냉동절단', '생강', '절단', '편마늘'..."
4,서울 동작구 성대로 24 여기로,찹스테이크,[],"{'시판돈까스소스스테이크소스', '파프리카색별로', '브로콜리', '어린잎채소', ..."


In [102]:
menu_and_ingred['재료'][2]

"{'참기름', '고추장', '설탕', '붉은고추', '식초', '매실', '매실액', '청양고추', '새우젓', '고춧가루', '마늘', '깨소금', '깨', '골뱅이', '다진마늘', '레몬즙', '간장', '부추'}"

In [98]:
recipe[recipe['음식']=='고르곤졸라피자']['재료'].item()

"{'시판용', '간마늘', '시금치또띠아', '홍고추', '바질', '배', '후추', '베샤멜소스', '모짜렐치즈', '소금', '모짜렐라', '모짜렐라치즈', '토르티야', '슬라이스아몬드', '물', '생크림', '핫소스', '버터', '고르곤졸라치즈', '베이컨', '고르곤졸라치즈피칸테', '블루치즈', '다진', '참기름', '양파', '꿀', '올리브오일', '아몬드슬라이스', '슬라이스', '아몬드가루', '피칸테', '피망', '파아슬리가루', '파슬리가루', '설탕', '여러가지견과류', '견과류', '치즈피자', '마늘', '블랙올리브', '고르곤졸라', '피자치즈', '올리고당', '밀가루', '또띠아제일', '슈레드모짜렐라피자치즈', '아몬드', '파슬리', '다진마늘', '통마늘개', '이스트', '또띠아', '유자청', '블루베리', '우유'}"

In [ ]:
import re

def str_to_set(str1):
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+') # 한글과 띄어쓰기를 제외한 모든 글자
    
    result = hangul.sub('', str1)
    
    return list(result.split())

In [52]:
for i in range(len(menu_and_ingred)):
    tmp_set3 = menu_and_ingred['재료'][i]
    tmp_set3_list = str_to_set(tmp_set3)
    for men in tmp_set3_list:
        print(men)

살짝
마늘
생크림
꿀이진하면섞음
참기름
피칸테
파슬리
통마늘개
올리브오일
간마늘
조금
소금
바질
피자치즈
토르티야
고르곤졸라치즈
홍고추
꿀
버터
파슬리가루
원하는만큼
적당량
고르곤졸라
유자청
배부른
다진
설탕
중력분
만두
또띠아
양파
모짜렐라치즈
치즈
호두
치즈피자
약간
블루치즈
배
시판용
이스트
또띠아제일
다진마늘
여러가지견과류
모짜렐치즈
블랙올리브
블루베리
올리고당
아몬드가루
핫소스
견과류
밀가루
모짜렐라
슬라이스
물
아몬드
피망
파아슬리가루
레몬즙
머스터드
햄
방울토마토
로메인
토마토
삶은
달걀
바질
파마산치즈반컵씩
파마산치즈가루
들기름
파마산
중간크기
이탈리안
치즈가루
마요네즈
치커리
설탕
후춧가루
머스타드
덩어리
빵가루
두꺼운
식초
계란
양파
파마산치즈
굴
플레인
약간
디죤
베이컨
식용유
로메인상추
올리브유
다진마늘
식빵
플레인요거트
샐러드
레몬쥬스
우스터소스
올리고당
양상추
상추
슬라이스
멸치액젓
향신료
노른자
아몬드
후추
요거트
또는
튀김가루
안초비
편으로
마늘
레몬즙
골뱅이
간장
양념
매실
청양고추
부추
참기름
파프리카
다진마늘
붉은고추
자름
적
고추장
깨
새우젓
고춧가루
설탕
진액
깨소금
혹은
식초
매실액
통조림
맞게
고운
굵은
냉동
참기름
꽈리고추
파프리카
피쉬아울렛절단낙지
애호박
낙지데친물
간마늘
붉은고추
꼬집
삶은
숙주
굵은소금
생강가루
원하는만큼
적당량
미향
풋고추
소금기호에
갈은
식물성기름
청고추
깨소금
양념장
양파
청양
찹쌀전분
고추가루
맛술
게장간장
떡국떡
청양고춧가루
식용유
콩나물무침
개
다진마늘
고운고춧가루
생강청
농도에맞게
그냥
삶은계란
콩나물
천연조미료해물
연두
고추기름
밀가루숟가락
버섯
멸치
청상추
토치
피망
육수
중사이즈
편마늘
호박
새우
통마늘
마늘
찹쌀
떡볶이떡
표고버섯
쥬키니
매실청
참기름숟가락
간생강
목이버섯
당근
양파중간크기
만능양념장
사과즙
양배추
떡
들기름
파
꿀
낙지
전분물
깨
냉동절단
중간크기
다진생강
고춧가루
팽이
후추가루
소주
볶은깨
매운고춧가루
후춧가루
껍질
땡초
낙지삶은물
한
훈제파프리카가루
썬
쪽

In [56]:
from konlpy.tag import Komoran
okt=Komoran()

for i in range(len(menu_and_ingred)):
    tmp_set3 = menu_and_ingred['재료'][i]
    tmp_set3_list = str_to_set(tmp_set3)
    for men in tmp_set3_list:
        print(okt.nouns(men))

[]
['마늘']
['크림']
['이진하']
['참기름']
['피칸', '테']
['파슬리']
['통', '마늘', '개']
['올리브', '오일']
['간', '마늘']
[]
['소금']
['바질']
['피자', '치즈']
['토르티야']
['고르곤졸라', '치즈']
['홍', '고추']
[]
['버터']
['파슬리', '가루']
['만큼']
['적', '당량']
['고르곤졸라']
['유자', '청']
[]
[]
['설탕']
['중력', '분']
['만두']
[]
['양파']
['모', '짜', '레', '라', '치즈']
['치즈']
['호두']
['치즈', '피자']
[]
['블루', '치즈']
['배']
['시판']
['이스트']
[]
['마늘']
['가지', '견과', '류']
['모', '짜', '레', '치즈']
['블랙', '올리브']
['블루베리']
[]
['아몬드', '가루']
['핫소스']
['견과', '류']
['밀가루']
['모', '짜', '레']
['이', '이스']
['물']
['아몬드']
['피망']
['파', '아스', '리', '가루']
['레몬', '즙']
['머스터드']
['햄']
['방울토마토']
['로메']
['토마토']
['삶']
['달걀']
['바질']
['파', '마산', '치즈', '컵']
['파', '마산', '치즈', '가루']
['들기름']
['파', '마산']
['중간', '크기']
['이탈리안']
['치즈', '가루']
['마요네즈']
['치', '커리']
['설탕']
[]
['머스타드']
['덩어리']
['빵가루']
[]
['식초']
['계란']
['양파']
['파', '마산', '치즈']
['굴']
['플레인']
[]
['디', '죤']
['베이컨']
['식용유']
['로메', '상추']
['올리브', '유']
['마늘']
['식빵']
['플레인', '요거']
['샐러드']
['레몬', '쥬스']
['우스터', '소스']
[]
['상추']
['상추']
['이', '이스']
['멸치', '액', '젓'

In [57]:
tmp_set3 = menu_and_ingred['재료'][0]
tmp_set3_list = str_to_set(tmp_set3)

In [71]:
from konlpy.tag import Kkma

mecab = Kkma()

words = ["살짝", "마늘", "생크림", "꿀이진하면섞음", "참기름", "피칸테", "파슬리"]


In [81]:
mecab.pos("적당량")

[('적당량', 'NNG')]

In [77]:
tmp_set3

"{'살짝', '마늘', '생크림', '꿀이진하면섞음', '참기름', '피칸테', '파슬리', '통마늘개', '올리브오일', '간마늘', '조금', '소금', '바질', '피자치즈', '토르티야', '고르곤졸라치즈', '홍고추', '꿀', '버터', '파슬리가루', '원하는만큼', '적당량', '고르곤졸라', '유자청', '배부른', '다진', '설탕', '중력분', '만두', '또띠아', '양파', '모짜렐라치즈', '치즈', '호두', '치즈피자', '약간', '블루치즈', '배', '시판용', '이스트', '또띠아제일', '다진마늘', '여러가지견과류', '모짜렐치즈', '블랙올리브', '블루베리', '올리고당', '아몬드가루', '핫소스', '견과류', '밀가루', '모짜렐라', '슬라이스', '물', '아몬드', '피망', '파아슬리가루'}"

In [ ]:
def zacard(A,B):
    intersection_cardinality = len(set.intersection(*[set(A), set(B)]))
    union_cardinality = len(set.union(*[set(A), set(B)]))
    similar = intersection_cardinality / float(union_cardinality)
    
    return similar

def find_ingred(menu_name,recipe):
    main_score=0
    main_menu=""
    
    for menu in recipe['음식']:
        tmp_score = zacard(menu, menu_name)
        
        if tmp_score > main_score:
            main_score = tmp_score
            main_menu = menu
    return main_menu

def matching_menu_ingred(crawled_menu, recipe):
    #레시피 불러오기
    adr=""
    recipe = pd.read_csv(adr+'통합레시피_유니크.csv',encoding = 'euc-kr')
    ingred_list =[]

    for i in range(len(crawled_menu)):
    
        tmp1 = find_ingred(crawled_menu['메뉴이름'][i],recipe)
        ingredients = recipe[recipe['음식']==tmp1]['재료'].item()
        tmp_set = set()
        
        for ingredient in ingredients:
            if check_fd(ingredient):
                tmp_set.add(ingredient)
                
        ingred_list.append(tmp_list)

    ingred_list=pd.DataFrame({'재료':ingred_list})
    crawled_menu['재료'] = ingred_list
    return crawled_menu

def check_fd(ingredient):
    

def crawling_and_matching(df, recipe):
    crawled = crawling_menus(df)
    
    result =matching_menu_ingred(crawled, recipe)
    
    return result

# 1. 음식 이름 남기기
# 2. str -> 리스트or set으로 바꿔주는 함수 짜기
# 3. 외래어 또는 아예 영어 번역

In [38]:
test_str = menu_and_ingred['재료'][0]

In [47]:
#2.
import re

def str_to_set(str1):
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+') # 한글과 띄어쓰기를 제외한 모든 글자
    
    result = hangul.sub('', str1)
    
    return list(result.split())

In [48]:
test_set = str_to_set(test_str)

In [49]:
test_set[0]

'마늘'

In [ ]:
test

In [107]:
zacard('설맥홍대서교점','설맥 홍대 서 교 점')

0.875